In [153]:
import os
import json

import pandas as pd
import numpy as np
import openai

df = pd.read_csv('output.csv')

In [154]:
df = df[df['comments'].apply(lambda x: len(x) > 3)]
df

,number,title,writer,is_open,body,comments
0,1214,Memory leak of 150MB per proof,mitschabaude,open,Reported on discord:\r\n![image](https://githu...,"[('dfstio', ""It was tested using the code in h..."
5,1207,Enable compiling smart contracts along with al...,mitschabaude,open,"**Why**: Currently, `zk deploy` fails if the c...","[('dfstio', 'The better solution would be to e..."
11,1189,"Proof.fromJSON() for times in a process, faile...",coldstar1993,open,I hv a zkProgram named `JoinSplitProver `and i...,"[('Comdex', 'This seems to be related to the w..."
13,1185,Expose dummyBase64Proof(),maht0rz,closed,Would it be possible to expose `dummyBase64Pro...,"[('mitschabaude', 'No reason, PR welcome!'), (..."
15,1174,Remove Poseidon from PLUME [ERC 7524 incompati...,Divide-By-0,closed,We noticed that you deviated from [ERC 7524](h...,"[('Trivo25', 'Thanks for raising this issue! W..."
...,...,...,...,...,...,...
647,18,Ensure tree shaking is used during the build,jasongitmail,closed,We need to get the client bundle size _much_ s...,"[('mitschabaude', 'I already poked into this a..."
648,17,Convert the build to output ES6,jasongitmail,closed,Todo: Check with Nathan & Martin to ask about ...,"[('mitschabaude', ""Initial progress: I tried t..."
649,16,Get build to work on the client,jasongitmail,closed,NaN,"[('mitschabaude', ""I'm not sure what this was ..."
650,15,Change the build to output 4 artifacts,jasongitmail,closed,- prover.js\r\n- prover.key\r\n- verifier.js \...,"[('bkase', ""Note the verifier.js/verifier.key ..."


In [155]:
split_df = np.array_split(df, len(df))
split_df[129]

/Users/berkingurcan/opt/anaconda3/envs/py39/lib/python3.9/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


,number,title,writer,is_open,body,comments
236,719,investigate and improve apple silicon performance,Trivo25,closed,"during the last office hours, we have received...","[('jasongitmail', ""@Trivo25 If you're able to ..."


In [156]:
SUMMARIZE_PROMPT = '''You are a professional MINA zkApp Developer. 
    The data is about Github Issues and their comments. 
    Understand the issue correctly. If solution is in the comments column deduct the solution
    explain it in detail to junior MINA zkApp Developer who has this problem. 
    If solution is solved by Pull Request include the PR number as a solution.
    If solution includes external link, provide it to user as a solution.
    If solution is not clear or not found in comments, Try to produce some solution suggestions.
    So, show the solution as the format:
    Issue: issue body 
    Solution: 
    Cause of the problem and detailed solution and explanation of the problem'''

FULL_PROMPT = '''You are a professional MINA zkApp Developer. 
    The data is about Github Issues and their comments. 
    Understand the issue correctly. If solution is in the comments column deduct the solution. 
    Do not summarize the solution comment. We need all data, code examples etc. for the solution.
    If solution is solved by Pull Request include the PR number as a solution.
    If solution includes external link, provide it to user as a solution.
    If solution is not clear or not found in comments, Try to produce some solution suggestions.
    So, show the solution like:
        Issue: Issue body
        Solution: Cause of the problem and detailed solution and explanation of the problem
    '''

STORE_FORMAT = '''You are a professional MINA zkApp Developer. 
    The data is about Github Issues and their comments. 
    Understand the issue correctly. If solution is in the comments column deduct the solution. 
    Do not summarize the solution comment. We need all data, code examples etc. for the solution.
    If solution is solved by Pull Request include the PR number as a solution.
    If solution includes external link, provide it to user as a solution.
    If solution is not clear or not found in comments, Try to produce some solution suggestions.
    So, export as a json object the issue solution like:
    {issue: Issue body,
    solution: Cause of the problem and detailed solution and explanation of the problem} '''

In [157]:
## Tools
import json

from typing import Type
from pydantic import BaseModel, Field
from langchain.tools import BaseTool

class JsonExporterInput(BaseModel):
    issue: str = Field(description="issue body")
    solution: str = Field(description="solution content")

class JsonExporterTool(BaseTool):
    name = "export_json"
    description = """takes issue and solution and exports it as json"""
    
    args_schema: Type[BaseModel] = JsonExporterInput

    def _run(self, issue: str, solution: str):

        json_object = {
            "issue": issue,
            "solution": solution
        }
        print("heyo: ", json.dumps(json_object))
        output.append(json.dumps(json_object))
        return json.dumps(json_object)

In [158]:
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_types import AgentType

from langchain.llms import OpenAI

OPENAI_API_KEY = ''

def export_issue(index):
    llm = ChatOpenAI(temperature=0.1, openai_api_key=OPENAI_API_KEY, model="gpt-3.5-turbo-0613")
    llmz = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY )
    tools = [JsonExporterTool()]
    
    agent = create_pandas_dataframe_agent(
        llm,
        split_df[index], 
        verbose=True,
        agent_type=AgentType.OPENAI_FUNCTIONS,
    )
    
    output = agent.run(
        FULL_PROMPT,
    )

    return output



In [159]:
xyz = export_issue(129)
xyz



> Entering new AgentExecutor chain...


Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: HTTPSConnectionPool(host='api.openai.com', port=443): Max retries exceeded with url: /v1/chat/completions (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7fe8fc246c70>: Failed to resolve 'api.openai.com' ([Errno 8] nodename nor servname provided, or not known)")).
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: HTTPSConnectionPool(host='api.openai.com', port=443): Max retries exceeded with url: /v1/chat/completions (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7fe8fc246340>: Failed to resolve 'api.openai.com' ([Errno 8] nodename nor servname provided, or not known)")).
Retrying langchain.chat_models.openai.

KeyboardInterrupt: 